In [ ]:
# 練習

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
# from torchvision import datasets, transforms
import dataset
import mainmodel
import numpy as np
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve

In [2]:

Dataset = dataset.dataset("Obrid_AE", "data")
Dataset.concat_data("sample_data",500)
Dataset = dataset.dataset("Obrid_AE", "test")
print("----------------------")
Dataset.concat_data("sample_test",100)
print("----------------------")
data = Dataset.read_savedata("sample_test")
print(data.shape[0])
print("----------------------")
data, test_data , anomaly_data= Dataset.read_traindata("sample_data", "sample_test", 1000, 256, 1)


(500, 256)
(500, 256)
----------------------
(100, 256)
(100, 256)
----------------------
(100, 256)
100
----------------------
(500, 256)
(100, 256)
1148 0
rate 0.9
data.shape[0]: 500
rate 450
TrainData (1000, 256, 1, 1, 256)
TestData (50, 256)
ÄnomalyDta (100, 256)


In [4]:

losslist=[]

model = mainmodel.Autoencoder2()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

outputs = []
#テンソル型に変換
data = torch.from_numpy(data.astype(np.float32)).clone()
for epoch in data:
    for data in epoch:
        recon = model(data)
#         print(recon.shape, data.shape)
        loss = criterion(recon, data)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    print('Loss:{:.4f}'.format(float(loss)))
    losslist.append(loss)
    outputs.append((epoch, data, recon))

Loss:0.0160
Loss:0.0064
Loss:0.0027
Loss:0.0052
Loss:0.0045
Loss:0.0029
Loss:0.0031
Loss:0.0019
Loss:0.0015
Loss:0.0016
Loss:0.0030
Loss:0.0021
Loss:0.0024
Loss:0.0017
Loss:0.0027
Loss:0.0035
Loss:0.0011
Loss:0.0019
Loss:0.0016
Loss:0.0009
Loss:0.0010
Loss:0.0014
Loss:0.0009
Loss:0.0021
Loss:0.0009
Loss:0.0013
Loss:0.0013
Loss:0.0013
Loss:0.0006
Loss:0.0012
Loss:0.0012
Loss:0.0013
Loss:0.0011
Loss:0.0014
Loss:0.0013
Loss:0.0007
Loss:0.0008
Loss:0.0011
Loss:0.0053
Loss:0.0008
Loss:0.0007
Loss:0.0008
Loss:0.0009
Loss:0.0012
Loss:0.0013
Loss:0.0008
Loss:0.0014
Loss:0.0012
Loss:0.0013
Loss:0.0006
Loss:0.0007
Loss:0.0006
Loss:0.0005
Loss:0.0004
Loss:0.0006
Loss:0.0010
Loss:0.0012
Loss:0.0012
Loss:0.0011
Loss:0.0016
Loss:0.0006
Loss:0.0011
Loss:0.0005
Loss:0.0010
Loss:0.0010
Loss:0.0010
Loss:0.0012
Loss:0.0008
Loss:0.0004
Loss:0.0013
Loss:0.0011
Loss:0.0004
Loss:0.0016
Loss:0.0012
Loss:0.0011
Loss:0.0009
Loss:0.0011
Loss:0.0010
Loss:0.0012
Loss:0.0011
Loss:0.0007
Loss:0.0012
Loss:0.0008
Loss

In [10]:
model_name = "newmodel"
ModelEdit = mainmodel.Modeledit("syuron")
ModelEdit.save_model(model, model_name) 

In [3]:

Dataset = dataset.dataset("Obrid_AE", "data")
Dataset.concat_data("sample_data",500)
Dataset = dataset.dataset("Obrid_AE", "test")
print("----------------------")
Dataset.concat_data("sample_test",100)
print("----------------------")
data = Dataset.read_savedata("sample_test")
print(data.shape[0])
print("----------------------")
data, test_data , anomaly_data= Dataset.read_traindata("sample_data", "sample_test", 1000, 256, 1)


(500, 256)
(500, 256)
----------------------
(100, 256)
(100, 256)
----------------------
(100, 256)
100
----------------------
(500, 256)
(100, 256)
1148 0
rate 0.9
data.shape[0]: 500
rate 450
TrainData (1000, 256, 1, 1, 256)
TestData (50, 256)
ÄnomalyDta (100, 256)


In [4]:
import dataloader
input_model = mainmodel.Autoencoder2()
input_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_model = mainmodel.Autoencoder2().to(device)
input_model.load_state_dict(torch.load("/Users/yukihorikawa/Desktop/LAB_LAST/AutoEncoder/program/syuron/model_data/20211203/newmodel.pth", map_location=device))

recon_list, encoded_list, input_list = dataloader.ocsvm_dataset(input_model, test_data)

NameError: name 'model' is not defined

In [8]:
import dataloader
num = 1000
input_model = mainmodel.Autoencoder2()
input_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_model = mainmodel.Autoencoder2().to(device)
input_model.load_state_dict(torch.load("/Users/yukihorikawa/Desktop/LAB_LAST/AutoEncoder/program/Obrid_AE/model_data/20211202/epoch1000.pth", map_location=device))
# input_model = input_model.to(device)
# Dataset.savenumpy(data, test_data, anomaly_data, num)

new_model = mainmodel.Modeledit("Obrid_AE").read_model(input_model,"Obrid_AE/model_data/20211202/epoch1000.pth")
# recon_list, encoded_list, input_list = dataloader.ocsvm_dataset(new_model, data)

input_list = []
recon_list = []
encoded_list = []
dataLen = data.shape[0]
for i in range(dataLen):
    input =  torch.from_numpy((data[i]).astype(np.float32)).clone()
    input = input[np.newaxis, np.newaxis, :]
    recon = new_model(data).detach().numpy()
    encoded = new_model.encoder(input)

    input_list.append(input)
    recon_list.append(recon)
    encoded_list.append(encoded)

    if ((dataLen/i)*100) % 10 == 0:
        print("--------{}/{}--------".format(i,dataLen))

TypeError: '_IncompatibleKeys' object is not callable

In [13]:
for i in range(data.shape[0]):
    input_list = []
    recon_list = []
    encoded_list = []
    dataLen = data.shape[0]
    for i in range(dataLen):
        input =  torch.from_numpy((test_data[i]).astype(np.float32)).clone()
        input = input[np.newaxis, np.newaxis, :]
        recon = model(input).detach().numpy()
        encoded = model.encoder(input)

        input_list.append(input)
        recon_list.append(recon)
        encoded_list.append(encoded)

        if ((dataLen/i)*100) % 10 == 0:
            print("--------{}/{}--------".format(i,dataLen))

ZeroDivisionError: division by zero

In [ ]:
for i in range(anomaly_data.shape[0]):
    # if i % 10 == 0:
    data0 =  torch.from_numpy((test_data[i]).astype(np.float32)).clone()
    test0 =  torch.from_numpy((anomaly_data[i]).astype(np.float32)).clone()
    print(type(data0))
    plt.figure(figsize=(9, 2))
    # data = data0.numpy()
    data = data0[np.newaxis, np.newaxis, :]
    recon = input_model(data).detach().numpy()
    plt.plot(data0.numpy())
    plt.plot(recon.flatten())
    plt.show()
    
    anomalypoint=sum(abs(recon.flatten()-data0.numpy()))
    print("normal:"+str(anomalypoint))
    if(anomalypoint>10):
        print("anomaly")
        
    plt.figure(figsize=(9, 2))
    # data = test0.numpy()
    data = test0[np.newaxis, np.newaxis, :]
    recon = input_model(data).detach().numpy()
    plt.plot(test0.numpy(),label="testdata")
    plt.plot(recon.flatten(),label="recondata")
    plt.show()
    
    anomalypoint=sum(abs(recon.flatten()-test0.numpy()))
    print("anomarly:"+str(anomalypoint))
    if(anomalypoint>10):
        print("anomaly")

In [ ]:
for i in range(anomaly_data.shape[0]):
    # if i % 10 == 0:
    data0 =  torch.from_numpy((test_data[i]).astype(np.float32)).clone()
    test0 =  torch.from_numpy((anomaly_data[i]).astype(np.float32)).clone()
    print(type(data0))
    plt.figure(figsize=(9, 2))
    # data = data0.numpy()
    data = data0[np.newaxis, np.newaxis, :]
    recon = input_model(data).detach().numpy()
    plt.plot(data0.numpy())
    plt.plot(recon.flatten())
    plt.show()
    
    anomalypoint=sum(abs(recon.flatten()-data0.numpy()))
    print("normal:"+str(anomalypoint))
    if(anomalypoint>10):
        print("anomaly")
        
    plt.figure(figsize=(9, 2))
    # data = test0.numpy()
    data = test0[np.newaxis, np.newaxis, :]
    recon = input_model(data).detach().numpy()
    plt.plot(test0.numpy(),label="testdata")
    plt.plot(recon.flatten(),label="recondata")
    plt.show()
    
    anomalypoint=sum(abs(recon.flatten()-test0.numpy()))
    print("anomarly:"+str(anomalypoint))
    if(anomalypoint>10):
        print("anomaly")

In [18]:
data.shape

(1000, 256, 1, 1, 256)

In [20]:
data

array([[[[[0.0174216 , 0.01567944, 0.01393728, ..., 0.22299652,
           0.22125436, 0.2195122 ]]],


        [[[0.00174216, 0.00174216, 0.00174216, ..., 0.06968641,
           0.06968641, 0.06271777]]],


        [[[0.01393728, 0.01045296, 0.01393728, ..., 0.12369338,
           0.12543554, 0.12195122]]],


        ...,


        [[[0.01393728, 0.01045296, 0.01393728, ..., 0.12369338,
           0.12543554, 0.12195122]]],


        [[[0.00348432, 0.00522648, 0.00696864, ..., 0.12195122,
           0.12020906, 0.11672474]]],


        [[[0.00522648, 0.00348432, 0.00522648, ..., 0.08885017,
           0.09059233, 0.08362369]]]],



       [[[[0.02264808, 0.01916376, 0.02264808, ..., 0.14634146,
           0.1445993 , 0.1445993 ]]],


        [[[0.0174216 , 0.01219512, 0.01219512, ..., 0.21254355,
           0.21080139, 0.20905923]]],


        [[[0.00348432, 0.00174216, 0.00348432, ..., 0.09059233,
           0.09059233, 0.08536585]]],


        ...,


        [[[0.01567944, 0.0139372

In [ ]:
Dataset = dataset.dataset("Obrid_AE", "test")
dataset.

In [19]:
test_data.shape

(50, 256)